In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

## Training

Download some data, for example: [http://mattmahoney.net/dc/text8.zip](http://mattmahoney.net/dc/text8.zip)

In [19]:
import word2vec

Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

This will create a `text8-phrases` that we can use as a better input for `word2vec`.
Note that you could easily skip this previous step and use the origial data as input for `word2vec`.

Train the model using the `word2phrase` output.

In [20]:
word2vec.word2vec('result.txt', 'ress.bin', size=100, verbose=True)

Starting training using file result.txt
Vocab size: 667
Words in train file: 11837


That generated a `text8.bin` file containing the word vectors in a binary format.

Do the clustering of the vectors based on the trained model.

In [21]:
word2vec.word2clusters('ress.bin', 'result-clusters.txt', 100, verbose=True)

Starting training using file ress.bin
Vocab size: 4
Words in train file: 1713


That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [22]:
import word2vec

Import the `word2vec` binary file created above

In [23]:
model = word2vec.load('ress.bin')

We can take a look at the vocabulaty as a numpy array

In [24]:
model.vocab

array([u'</s>', u'#EVET', u'i\xe7in', u'#Evet', u've', u'bu',
       u'#MemurSeneDavetTercihEvet', u'#evet',
       u'#Hay\u0131rDemiyoruz\xc7\xdcNK\xdc', u'T\xfcrkiye', u'bir',
       u'aday', u'G\xfc\xe7l\xfc', u'de', u'Bu', u'16', u'ne',
       u'#HaydiT\xfcrkiye', u'1', u'b\xfcy\xfck', u'g\xfc\xe7l\xfc',
       u'Hem', u'@TurkayMetin:', u'y\xfczden', u'#HAYIR', u'Referandumda',
       u'!!', u'kar\u015f\u0131', u'olmaya', u'\xe7\u0131k\u0131nca',
       u'\xe7at\u0131', u'\xf6nerirler', u'ak\u015feneri', u'\u0130zleyin',
       u'i\xe7in,', u'\xe7\u0131kt\u0131klar\u0131',
       u'ba\u015fkanl\u0131\u011fa', u'meral', u'ikisi', u'utan\u0131r',
       u'istikrarl\u0131', u'\xe7\u0131kar\u0131p', u'heralde',
       u'@ReisciHatice:', u'@disli_merve86:', u',', u'karar\u0131m\u0131z',
       u'kadar', u'@rali35:', u'\u0130\xe7in', u'oyunuz',
       u'@osmanll_beyi:', u'rt', u'http\u2026RT', u'(Oy',
       u's\xfcrecektir.', u'hafta', u'Anket', u'??', u'olacakt\u0131r',
       u'ANKETT

Or take a look at the whole matrix

In [25]:
model.vectors.shape

(667, 100)

In [26]:
model.vectors

array([[ 0.14333282,  0.15825513, -0.13715845, ...,  0.05456942,
         0.10955409,  0.00693387],
       [ 0.06664074, -0.06769899,  0.10004526, ...,  0.10045119,
        -0.06882237, -0.02533416],
       [ 0.05349666, -0.05904634,  0.09227172, ...,  0.09860691,
        -0.07739629, -0.01857724],
       ..., 
       [ 0.02696064,  0.0314932 ,  0.02478098, ..., -0.14323677,
         0.11688532, -0.10754634],
       [ 0.06302015, -0.05156104,  0.09801275, ...,  0.09141515,
        -0.06747776, -0.01908772],
       [ 0.04681491, -0.06786318,  0.09205017, ...,  0.0919707 ,
        -0.07321247, -0.02706222]])

We can retreive the vector of individual words

In [27]:
model['idam'].shape

(100,)

In [28]:
model['idam'][:10]

array([ 0.05424013, -0.06254178,  0.09455197, -0.08326341, -0.05127008,
       -0.03213024, -0.02676981, -0.02153507,  0.02765418,  0.01497025])

We can do simple queries to retreive words similar to "socks" based on cosine similarity:

In [29]:
indexes, metrics = model.cosine('idam')
indexes, metrics

(array([ 69, 447, 191, 443,  85,  43, 292, 117,   8,   7]),
 array([ 0.99964538,  0.99963192,  0.99962978,  0.99962612,  0.99962009,
         0.99961879,  0.99961694,  0.99961638,  0.99961524,  0.9996137 ]))

This returned a tuple with 2 items:
1. numpy array with the indexes of the similar words in the vocabulary
2. numpy array with cosine similarity to each word

Its possible to get the words of those indexes

In [30]:
model.vocab[indexes]

array([u'da', u'@akkadn_26:', u'#hay\u0131r', u'#Akparti', u'Hay\u0131r',
       u'@ReisciHatice:', u'iradeye', u'daha',
       u'#Hay\u0131rDemiyoruz\xc7\xdcNK\xdc', u'#evet'], 
      dtype='<U78')

There is a helper function to create a combined response: a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [31]:
model.generate_response(indexes, metrics)

rec.array([(u'da', 0.9996453834842265), (u'@akkadn_26:', 0.9996319174433688),
 (u'#hay\u0131r', 0.9996297831994851), (u'#Akparti', 0.9996261227615416),
 (u'Hay\u0131r', 0.9996200872714502),
 (u'@ReisciHatice:', 0.9996187866042381), (u'iradeye', 0.9996169418796403),
 (u'daha', 0.999616378188327),
 (u'#Hay\u0131rDemiyoruz\xc7\xdcNK\xdc', 0.9996152368835952),
 (u'#evet', 0.9996137044813816)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

Is easy to make that numpy array a pure python response:

In [32]:
model.generate_response(indexes, metrics).tolist()

[(u'da', 0.9996453834842265),
 (u'@akkadn_26:', 0.9996319174433688),
 (u'#hay\u0131r', 0.9996297831994851),
 (u'#Akparti', 0.9996261227615416),
 (u'Hay\u0131r', 0.9996200872714502),
 (u'@ReisciHatice:', 0.9996187866042381),
 (u'iradeye', 0.9996169418796403),
 (u'daha', 0.999616378188327),
 (u'#Hay\u0131rDemiyoruz\xc7\xdcNK\xdc', 0.9996152368835952),
 (u'#evet', 0.9996137044813816)]

### Phrases

Since we trained the model with the output of `word2phrase` we can ask for similarity of "phrases"

In [38]:
indexes, metrics = model.cosine('yetkilisi')
model.generate_response(indexes, metrics).tolist()

[(u'news', 0.9972785387251313),
 (u'yay\u0131mlanan', 0.9971057734839921),
 (u'telefonla', 0.9970603290236499),
 (u'konvoy', 0.9967531315324603),
 (u'yetkilisi', 0.996713066815481),
 (u'toplant\u0131n\u0131n', 0.9966796915519048),
 (u'verdi\u011fi', 0.9965941774654892),
 (u'ingiliz', 0.9965921817610062),
 (u'brown', 0.9965833434147313),
 (u'h\xfcseyin', 0.9965249465320618)]

### Analogies

Its possible to do more complex queries like analogies such as: `king - man + woman = queen` 
This method returns the same as `cosine` the indexes of the words in the vocab and the metric

In [39]:
model.generate_response(indexes, metrics).tolist()

[(u'news', 0.9972785387251313),
 (u'yay\u0131mlanan', 0.9971057734839921),
 (u'telefonla', 0.9970603290236499),
 (u'konvoy', 0.9967531315324603),
 (u'yetkilisi', 0.996713066815481),
 (u'toplant\u0131n\u0131n', 0.9966796915519048),
 (u'verdi\u011fi', 0.9965941774654892),
 (u'ingiliz', 0.9965921817610062),
 (u'brown', 0.9965833434147313),
 (u'h\xfcseyin', 0.9965249465320618)]

### Clusters

In [22]:
clusters = word2vec.load_clusters('result-clusters.txt')

ValueError: Some errors were detected !
    Line #3 (got 1 columns instead of 1)

We can see get the cluster number for individual words

In [39]:
clusters['dog']

NameError: name 'clusters' is not defined

We can see get all the words grouped on an specific cluster

In [40]:
clusters.get_words_on_cluster(90).shape

NameError: name 'clusters' is not defined

In [41]:
clusters.get_words_on_cluster(90)[:10]

NameError: name 'clusters' is not defined

We can add the clusters to the word2vec model and generate a response that includes the clusters

In [42]:
model.clusters = clusters

NameError: name 'clusters' is not defined

In [43]:
indexes, metrics = model.analogy(pos=['paris', 'germany'], neg=['france'], n=10)

KeyError: 'paris'

In [44]:
model.generate_response(indexes, metrics).tolist()

[(u'#Hay\u0131rDemiyoruz\xc7\xdcNK\xdc', 0.9999983076674978),
 (u'#EVET', 0.9999979252163659),
 (u'VE', 0.999997727445976),
 (u'15', 0.9999974709044099),
 (u'#EVET\u2026RT', 0.9999974676813989),
 (u'#evet', 0.9999973989631359),
 (u'i\xe7in', 0.9999973316027865),
 (u'diyoruz.', 0.9999972452130969),
 (u'B\u0130R', 0.9999971474304682),
 (u'@kkaadder:', 0.9999969153788237)]